Todo:
- čísla týdnů (52+53 atd)

In [2]:
import os

In [3]:
import pandas as pd

In [4]:
folder = "data_raw"

In [5]:
df = pd.read_csv(os.path.join(folder,"ifpi_raw.csv"))

In [6]:
df

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel
0,0,CZ - ALBUMS - TOP 100,1. týden 2006,1,Album,PETR KOLÁŘ,UNIVERSAL MUSIC
1,1,CZ - ALBUMS - TOP 100,1. týden 2006,2,Music Bar,CHINASKI,UNIVERSAL MUSIC
2,2,CZ - ALBUMS - TOP 100,1. týden 2006,3,Čím to je?!,NO NAME,UNIVERSAL MUSIC
3,3,CZ - ALBUMS - TOP 100,1. týden 2006,4,44 slavných písniček,NEDVĚDI,UNIVERSAL MUSIC
4,4,CZ - ALBUMS - TOP 100,1. týden 2006,5,Best Of 1988-2005,PAVOL HABERA & TEAM,UNIVERSAL MUSIC
...,...,...,...,...,...,...,...
496801,45,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,46,Naše cesty,MAREK ZTRACENÝ,SUPRAPHON
496802,46,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,47,Naše hriechy (feat. Emma Drobná),ADAM ĎURICA,UNIVERSAL MUSIC
496803,47,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,48,Náš příběh (feat. Marek Lambora),TEREZA MAŠKOVÁ,WARNER MUSIC
496804,48,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,49,Patříme k sobě,OLGA LOUNOVÁ,OLGA LOUNOVÁ RECORDS


In [7]:
hitparady = df['hitparada'].drop_duplicates().to_list()
hitparady

['CZ - ALBUMS - TOP 100',
 'CZ - RADIO - TOP 100',
 'CZ - RADIO - TOP 20 MODERN ROCK',
 'CZ - ALBUMS CLASSICAL - TOP 10',
 'CZ - SINGLES DIGITAL - TOP 100',
 'CZ - SINGLES DIGITAL - TOP 50  CZ',
 'SK - ALBUMS - TOP 100',
 'SK - RADIO - TOP 100',
 'SK - RADIO - TOP 50 SK',
 'SK - SINGLES DIGITAL - TOP 100',
 'SK - SINGLES DIGITAL - TOP 50 SK',
 'CZ - RADIO - TOP 50 CZ']

## Opravy a čištění

### Aby byla data strojově čitelná

In [8]:
df['tyden'] = df['obdobi'].apply(lambda x: x.split(". t")[0] if ". t" in x else None)

In [9]:
df['mesic'] = df['obdobi'].apply(lambda x: x.split("/")[0].split(" ")[1] if "Měsíc" in x else None)

In [10]:
df['rok'] = df['obdobi'].apply(lambda x: x[-4:])

In [11]:
df.groupby('tyden')['track/album'].nunique().nsmallest(15)

tyden
5152       188
200903     190
200809     198
50.+51     206
53         328
52.+53     487
27.+28     519
30.+31     519
52.+01     524
52        1036
51        1785
51.+52    2072
50        4002
1         4108
31        4108
Name: track/album, dtype: int64

In [12]:
df[(df['hitparada'] == 'CZ - RADIO - TOP 100') & (df['rok'] == '2023')]

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel,tyden,mesic,rok
166750,0,CZ - RADIO - TOP 100,1. týden 2023,1,Forget Me,LEWIS CAPALDI,UNIVERSAL MUSIC,1,None,2023
166751,1,CZ - RADIO - TOP 100,1. týden 2023,2,Celestial,ED SHEERAN,WARNER MUSIC,1,None,2023
166752,2,CZ - RADIO - TOP 100,1. týden 2023,3,Perfect Melody,JONAS BLUE & JULIAN PERRETTA,UNIVERSAL MUSIC,1,None,2023
166753,3,CZ - RADIO - TOP 100,1. týden 2023,4,Vedle tebe usínám,MIRAI,UNIVERSAL MUSIC,1,None,2023
166754,4,CZ - RADIO - TOP 100,1. týden 2023,5,Anti-Hero,TAYLOR SWIFT,UNIVERSAL MUSIC,1,None,2023
...,...,...,...,...,...,...,...,...,...,...
171545,95,CZ - RADIO - TOP 100,Měsíc 9/2023,96,YEAH (feat. ClockClock),GLOCKENBACH & JOEL CORRY & TENCHI,UNIVERSAL MUSIC,None,9,2023
171546,96,CZ - RADIO - TOP 100,Měsíc 9/2023,97,Bad Memories (feat. Elley Duhé & FAST BOY),MEDUZA & JAMES CARTER,UNIVERSAL MUSIC,None,9,2023
171547,97,CZ - RADIO - TOP 100,Měsíc 9/2023,98,Do nebe se propadám,KRYŠTOF,I.RICHMOND.P./UNIVERSAL MUSIC,None,9,2023
171548,98,CZ - RADIO - TOP 100,Měsíc 9/2023,99,Hold My Hand (Music From Top Gun: Maverick),LADY GAGA,UNIVERSAL MUSIC,None,9,2023


In [13]:
df.sample(5)

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel,tyden,mesic,rok
381362,32,SK - RADIO - TOP 50 SK,18. týden 2023,33,Verím,KATARÍNA KNECHTOVÁ,SUKUBA AGENCY,18,None,2023
326145,76,SK - RADIO - TOP 100,3. týden 2018,77,The Spectre,ALAN WALKER,SONY MUSIC,3,None,2018
52700,50,CZ - ALBUMS - TOP 100,Měsíc 3/2020,51,Smutku dávám sbohem,HARLEJ,WARNER MUSIC,None,3,2020
385267,87,SK - SINGLES DIGITAL - TOP 100,17. týden 2017,88,Hard Times,PARAMORE,WARNER MUSIC,17,None,2017
193400,5,CZ - ALBUMS CLASSICAL - TOP 10,30. týden 2013,6,Bohemian Impressions,ČF/BĚLOHLÁVEK;NEUMANN;PEŠEK,SUPRAPHON,30,None,2013


### Oprava chybných dat

In [14]:
df["rok"] = df["rok"].apply(lambda x: x.replace("223","2023").replace(" 201","2010").replace("2104","2014").strip())

### Pryč s velkými písmeny (nebo jejich většinou)

In [16]:
df['interpret'] = df['interpret'].apply(lambda x: ' '.join(word.capitalize() for word in str(x).split()))
df['vydavatel'] = df['vydavatel'].apply(lambda x: ' '.join(word.capitalize() for word in str(x).split()))

In [17]:
df.sample(5)

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel,tyden,mesic,rok
341959,90,SK - RADIO - TOP 100,Měsíc 8/2020,91,Srdce nehasnou,Karel Gott/charlotte Ella Gottová,Supraphon,None,8,2020
181350,6,CZ - RADIO - TOP 20 MODERN ROCK,22. týden 2016,7,Ring Of Fire,Dragonforce,Nan,22,None,2016
177141,11,CZ - RADIO - TOP 20 MODERN ROCK,11. týden 2012,12,Tonight,Seether,Nan,11,None,2012
140772,22,CZ - RADIO - TOP 100,45. týden 2018,23,Minulost,Marek Ztracený,Supraphon,45,None,2018
376074,44,SK - RADIO - TOP 50 SK,31. týden 2021,45,Remember Me,Martin Harich,Universal Music,31,None,2021


### Hlavní intepreti

In [25]:
df[df['interpret'].str.contains(" A ")]['interpret'].drop_duplicates()

86                  Jaroslav Uhlíř A Zdeněk Svěrák
390                          Hana A Petr Ulrychovi
874                                  Mňága A Žďorp
9590                          Ryvolovi Wabi A Miky
12990                               Žalman A Spol.
14050                 Robert Křesťan A Druhá Tráva
17784          Havelka Ondřej A Jeho Melody Makers
21084                      Michnová Zuzana A Hosté
23460                   Wabi Daněk A Ďáblovo Stádo
24147                  Petr Hapka A Michal Horáček
25135          Ondřej Havelka A Jeho Melody Makers
26579                          Daniel Landa A Jiní
28567                  Věra Špinarová A Petra Janů
30456                           Laura A Její Tygři
34097                     Michal Prokop A Framus 5
58233                       Y.simonová A M.chladil
60492                Jan Vodňanský A Haňa Navarová
76104                             System Of A Down
88042                                Marek A Kamil
104510            David Michal 

In [26]:
def kdojeprvni(x):
    delice = ["&","/","(feat.","feat.","Feat."]
    for d in delice:
        if d in x:
            x = x.split(d)[0]
    return x.strip()

In [27]:
df['interpret_hlavni'] = df['interpret'].apply(lambda x: kdojeprvni(x))

In [28]:
df['interpret_hlavni'].drop_duplicates().to_list()

['Petr Kolář',
 'Chinaski',
 'No Name',
 'Nedvědi',
 'Pavol Habera',
 'Aneta Langerová',
 'Těžkej Pokondr',
 'Robbie Williams',
 'Damichi',
 'Divokej Bill',
 'Petr Novák',
 'Různí',
 'Vlasta Horváth',
 'Rammstein',
 'Madonna',
 'Petr Bende',
 'Helena Zeťová',
 'Jakub Smolík',
 'Čechomor',
 'Richard Müller',
 'Tatu',
 'Karel Gott',
 'Depeche Mode',
 'Petr Muk',
 'Lenka Filipová',
 'Michal David',
 'Black Eyed Peas',
 'Anastacia',
 'Juanes',
 'Amiri Ali',
 'Issa Sámer',
 'Tři Sestry',
 'Věra Špinarová',
 'Enya',
 'Karel Kryl',
 'Maxim Turbulenc',
 'Miro Žbirka',
 'Zuzana Navarová',
 'Iveta Bartošová',
 'Marie Rottrová',
 'Pražský Výběr',
 'Hana Hegerová',
 'Dan Bárta',
 'Iva Frühlingová',
 'Jiří Schelinger',
 'Leoš Mareš',
 'Janek Ledecký',
 'Eminem',
 'Jaroslav Uhlíř A Zdeněk Svěrák',
 'P.hapka',
 'Lucie Vondráčková',
 'Alanis Morissette',
 '4tet',
 'Jana Kirschner',
 'James Blunt',
 'Crazy Frog',
 'Lenka Dusilová',
 'Victoria',
 'Hana Zagorová',
 'Marika Gombitová',
 'Radůza',
 '2m',
 

In [29]:
len(df[(df['hitparada'] == 'CZ - RADIO - TOP 100')]['interpret_hlavni'].drop_duplicates().to_list())

1480

### Album/track

In [30]:
df['album'] = df.apply(lambda row: row['track/album'] if 'ALBUM' in row['hitparada'] else None, axis = 1)

In [31]:
df['track'] = df.apply(lambda row: row['track/album'] if 'ALBUM' not in row['hitparada'] else None, axis = 1)

In [32]:
df = df.drop(columns=['track/album','obdobi'])

In [33]:
poradi = ['hitparada','rok','mesic','tyden','pozice','album','track','interpret','interpret_hlavni','vydavatel']

In [34]:
df = df.reindex(columns=poradi)

### Seřazení

In [35]:
df = df.sort_values(by=['hitparada','rok','mesic','tyden','pozice'])

In [36]:
df.sample(10)

,hitparada,rok,mesic,tyden,pozice,album,track,interpret,interpret_hlavni,vydavatel
194020,CZ - ALBUMS CLASSICAL - TOP 10,2014,None,41,7,Inspirato,None,Yanni,Yanni,Sony Music
353802,SK - RADIO - TOP 100,2022,7,None,34,None,Slidin' (feat. Kodak Black),Jason Derulo,Jason Derulo,Warner Music
448651,CZ - RADIO - TOP 50 CZ,2007,None,14,26,None,Stromy,No Name,No Name,Universal Music
418813,SK - SINGLES DIGITAL - TOP 100,2022,None,51.+52,34,None,SPIT IN MY FACE!,Thxsomch,Thxsomch,Warner Music
148803,CZ - RADIO - TOP 100,2020,None,9,54,None,Body Back (feat. Maia Wright),Gryffin,Gryffin,Universal Music
458382,CZ - RADIO - TOP 50 CZ,2011,None,3,9,None,Všetko má svoj koniec,Rytmus Feat. Ego&tina,Rytmus,Warner Music
145365,CZ - RADIO - TOP 100,2019,None,32,16,None,I've Been Waiting (feat. Fall Out Boy),Lil Peep,Lil Peep,Sony Music
204888,CZ - SINGLES DIGITAL - TOP 100,2015,None,26,70,None,Marvin Gaye,Charlie Puth & Meghan Trainor,Charlie Puth,Warner Music
407201,SK - SINGLES DIGITAL - TOP 100,2021,None,2,22,None,HOLIDAY,Lil Nas X,Lil Nas X,Sony Music
378667,SK - RADIO - TOP 50 SK,2022,None,25,38,None,Otec,Para,Para,Sony Music


### Export do jednotlivých dataframů

In [37]:
import os

if not os.path.exists("data"):
    os.mkdir("data")
else:
    pass

In [38]:
for h in hitparady:
    df_p = df[df["hitparada"] == h]
    if "ALBUMS" in h:
        df_p = df_p.drop(columns=['track'])
    else:
        df_p = df_p.drop(columns=['album'])
    df_p2 = df_p[~df_p['mesic'].isnull()]
    h = h.replace(" ","_").replace("-_","")
    if len(df_p2) > 0:
        df_p2 = df_p2.drop(columns=['tyden'])
        df_p2.to_csv(os.path.join("data",f"{h}_mesicni.csv"))
    df_p2 = df_p[~df_p['tyden'].isnull()]
    if len(df_p2) > 0:
        df_p2 = df_p2.drop(columns=['mesic'])
        df_p2.to_csv(os.path.join("data",f"{h}_tydeni.csv"))